# Clustering: DTW (Dynamic Time Warping)

> Dynamic Time Warping (DTW) is a non-linear distance measure that computes the optimal alignment between two sequences (or point series),  
  allowing for stretching and compression of the time axis. It is especially useful for comparing sequences that are similar in shape but may be out of phase.  

The dataset:

We treat the following points as a sequence of spatial coordinates:

| Index | Point | Coordinates |
| ----- | ----- | ----------- |
| 0     | A     | (1, 1)      |
| 1     | B     | (2, 1)      |
| 2     | C     | (4, 3)      |
| 3     | D     | (5, 4)      |
| 4     | E     | (3, 4)      |

Let us consider the following two sequences of points (trajectories):  

Sequence 1: $A \rightarrow B \rightarrow C \rightarrow D$  
Sequence 2: $A \rightarrow B \rightarrow E \rightarrow D$  

**We want to compare these two sequences.**  

Even though they are similar overall, point $C$ is replaced by $E$ in Sequence 2 — and this may introduce misalignment in time/space.  


**Step-0: Euclidean Distance Matrix**  

We compute the pairwise Euclidean distances between points in both sequences.

| From \ To | A   | B   | E   | D   |
| --------- | --- | --- | --- | --- |
| A         | 0.0 | 1.0 | 3.6 | 5.0 |
| B         | 1.0 | 0.0 | 3.2 | 4.2 |
| C         | 3.6 | 2.8 | 1.4 | 1.4 |
| D         | 5.0 | 4.2 | 2.0 | 0.0 |

We use these distances as the cost of aligning one point from Sequence 1 with one point from Sequence 2.  

**Step-1: DTW Alignment Matrix**  

Let $S1 = [A, B, C, D]$ and $S2 = [A, B, E, D]$.

We compute the DTW cost matrix using dynamic programming:  

* Define a cost matrix $DTW[i][j]$ where:  
$$DTW[i][j] = \text{cost}(S1[i], S2[j]) + min(DTW[i−1][j], DTW[i][j-1], DTW[i-1][j-1])$$
* Initialization:  
  $DTW[0][0] = \text{dist}(A,A) = 0$
* Proceed to fill the matrix using the pairwise distances.  

**Step-2: Optimal Warping Path**  

Once the matrix is filled, we trace back the optimal warping path, which aligns points from Sequence 1 and 2 by minimizing the total distance.  

This path may repeat points (i.e., stretch) or skip certain alignments to accommodate shape differences.  

For instance:  

* $C$ in Sequence 1 may be aligned with both $E$ and $D$ in Sequence 2.  
* Even though $C \neq E$, DTW may align them because their distance is small $(1.4)$.  

**Step-3: DTW Distance**  

The DTW distance is the total cost along the optimal path.  

In our example:  

* Without DTW (naive Euclidean):  
  Total distance = $||A-A|| + ||B-B|| + ||C-E|| + ||D-D|| \approx 0 + 0 + 1.4 + 0 = 1.4$  

* With DTW (allowing warping), the alignment may stretch $C$ to better match $E$ and $D$, possibly lowering the effective cost by matching on shape rather than strict order.  

📌 **Conclusion: Why DTW Preserves Shape Similarity Across Sequences**   

| Property                 | Explanation                                                                   |
| ------------------------ | ----------------------------------------------------------------------------- |
| Time warping flexibility | Allows stretching/compression of sequences for better alignment               |
| Handles phase shifts     | Can align similar shapes even when shifted or distorted in time               |
| Sensitive to local shape | Preserves the sequence pattern better than Euclidean distance                 |
| Useful for time series   | Ideal for clustering and comparing curves (e.g., gestures, ECG, trajectories) |


In [6]:
# python3 -m pip install fastdtw
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
import numpy as np

x = np.array([[0,0], [1,1], [2,2]])
y = np.array([[0,0], [1,2], [2,3]])

distance, path = fastdtw(x, y, dist=euclidean)

print("FastDTW distance:", distance)
print("Path:", path)

FastDTW distance: 2.0
Path: [(0, 0), (1, 1), (2, 2)]
